In [14]:
# import time
# import requests
# import re
# import urllib.request
# from bs4 import BeautifulSoup
# from collections import deque
# from html.parser import HTMLParser
# from urllib.parse import urlparse
# import os
# 
# # headers = {
# #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
# #     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
# #     'Accept-Language': 'en-US,en;q=0.5',
# #     'Accept-Encoding': 'gzip, deflate, br',
# #     'Connection': 'keep-alive',
# #     'Upgrade-Insecure-Requests': '1',
# # }
# # 
# # 
# # # Function to get the hyperlinks from a URL
# # def get_hyperlinks(url):
# #     try:
# #         req = urllib.request.Request(url, headers=headers)
# #         with urllib.request.urlopen(req) as response:
# #             if not response.info().get('Content-Type').startswith("text/html"):
# #                 return []
# #             html = response.read().decode('utf-8')
# #     except Exception as e:
# #         print(e)
# #         return []
# # 
# #     parser = HyperlinkParser()
# #     parser.feed(html)
# #     return parser.hyperlinks
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# 
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# 
# # Initialize the Service with the path to the ChromeDriver executable
# service = Service(ChromeDriverManager().install())
# 
# # Pass the Service object to the WebDriver constructor
# driver = webdriver.Chrome(service=service, options=options)
# 
# def get_hyperlinks(url):
#     driver.get(url)
#     time.sleep(2)  # Adjust based on page load time
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     hyperlinks = [a['href'] for a in soup.find_all('a', href=True)]
#     return hyperlinks
# 
# 
# # Regex pattern to match a URL
# HTTP_URL_PATTERN = r'^http[s]*://.+'
# 
# # Class to parse the HTML and get the hyperlinks
# class HyperlinkParser(HTMLParser):
#     def __init__(self):
#         super().__init__()
#         # Create a list to store the hyperlinks
#         self.hyperlinks = []
# 
#     # Override the HTMLParser's handle_starttag method to get the hyperlinks
#     def handle_starttag(self, tag, attrs):
#         attrs = dict(attrs)
# 
#         # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
#         if tag == "a" and "href" in attrs:
#             self.hyperlinks.append(attrs["href"])
# 
# 
# # Function to get the hyperlinks from a URL that are within the same domain
# def get_domain_hyperlinks(local_domain, url):
#     clean_links = []
#     for link in set(get_hyperlinks(url)):
#         clean_link = None
# 
#     
#         # If the link is a URL, check if it is within the same domain
#         try: 
#             if re.search(HTTP_URL_PATTERN, link):
#                 # Parse the URL and check if the domain is the same
#                 url_obj = urlparse(link)
#                 if url_obj.netloc == local_domain:
#                     clean_link = link
#         except Exception as e:
#             print(e)
#                  
# 
#         # If the link is not a URL, check if it is a relative link
#         else:
#             try:
#                 if link.startswith("/"):
#                     link = link[1:]
#                 elif link.startswith("#") or link.startswith("mailto:"):
#                     continue
#                 clean_link = "https://" + local_domain + "/" + link
#             except Exception as e:
#                 print(e)
# 
#         if clean_link is not None:
#             try:
#                 if clean_link.endswith("/"):
#                     clean_link = clean_link[:-1]
#                 clean_links.append(clean_link)
#             except Exception as e:
#                 print(e)
# 
#     # Return the list of hyperlinks that are within the same domain
#     return list(set(clean_links))
# 
# def get_first_five_links(url):
#     seen = set()
#     queue = []
#     link_count = 0 
#     
#     for link in get_hyperlinks(url): 
#         try:
#             if link not in seen and link_count < 5: 
#                 queue.append(link)
#                 seen.add(link)
#                 link_count += 1 
#         except Exception as e:
#             print(f"Error processing link {link}: {e}")
#     
#     return queue[:5] 
# 
# 
# def create_directories_for_crawl(url):
#     # Parse the URL and get the domain
#     local_domain = urlparse(url).netloc
# 
#     # Create a directory to store the text files
#     if not os.path.exists("text/"):
#         os.mkdir("text/")
#     if not os.path.exists("text/" + local_domain + "/"):
#         os.mkdir("text/" + local_domain + "/")
# 
#     # Create a directory to store the csv files
#     if not os.path.exists("processed"):
#         os.mkdir("processed")
# 
# def crawl(url):
#     # Create a queue to store the URLs to crawl
#     queue = deque([url])
# 
#     # Create a set to store the URLs that have already been seen (no duplicates)
#     seen = {url}
# 
#     # Call create_directories_for_crawl with the initial URL to ensure directories are created
#     create_directories_for_crawl(url)
# 
#     # While the queue is not empty, continue crawling
#     while queue:
# 
#         # Get the next URL from the queue
#         url = queue.popleft()  # Changed from pop() to popleft() to maintain order
#         print(url)  # for debugging and to see the progress
#         try:
#             # Save text from the url to a <url>.txt file
#             save_page_content_to_file(url)
#         except Exception as e:
#             print(e)  
# 
#         try:    
#             # Get the first five hyperlinks from the URL and add them to the queue
#             first_five_links = get_first_five_links(url)
#             for link in first_five_links:
#                 if link not in seen:
#                     queue.append(link)
#                     seen.add(link)
#         except Exception as e:
#             print(e)
# 
# def save_page_content_to_file(url):
#     # Parse the URL and get the domain
#     local_domain = urlparse(url).netloc
# 
#     # Create directories to store the text files if they don't exist
#     create_directories_for_crawl(url)
# 
#     try:
#         # Get the text from the URL using BeautifulSoup
#         soup = BeautifulSoup(requests.get(url).text, "html.parser")
# 
#         # Get the text but remove the tags
#         text = soup.get_text()
# 
#         # If the crawler gets to a page that requires JavaScript, it will stop the crawl
#         if ("You need to enable JavaScript to run this app." in text):
#             print("Unable to parse page " + url + " due to JavaScript being required")
#         
#         # Write the text to the file in the text directory
#         with open('text/' + local_domain + '/' + url.replace("http://", "").replace("https://", "").replace("/", "_") + ".txt", "w") as f:
#             f.write(text)
#     except Exception as e:
#         print(e)
# 
# 


In [27]:
import time
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

# Initialize the Service with the path to the ChromeDriver executable
service = Service(ChromeDriverManager().install())

# Pass the Service object to the WebDriver constructor
driver = webdriver.Chrome(service=service, options=options)

def get_hyperlinks(url):
    driver.get(url)
    time.sleep(2)  # Adjust based on page load time
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    hyperlinks = [a['href'] for a in soup.find_all('a', href=True)]
    return hyperlinks


# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

# Define root domain to crawl
# domain = "ci.hartford.wi.us"
domain = "www.grainger.com"
# Full URL including the specific path to start crawling from
# full_url = "https://ci.hartford.wi.us/440/Experience-Downtown"
full_url = "https://www.grainger.com/"

# Class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])


# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None


        # If the link is a URL, check if it is within the same domain
        try: 
            if re.search(HTTP_URL_PATTERN, link):
                # Parse the URL and check if the domain is the same
                url_obj = urlparse(link)
                if url_obj.netloc == local_domain:
                    clean_link = link
        except Exception as e:
            print(e)


        # If the link is not a URL, check if it is a relative link
        else:
            try:
                if link.startswith("/"):
                    link = link[1:]
                elif link.startswith("#") or link.startswith("mailto:"):
                    continue
                clean_link = "https://" + local_domain + "/" + link
            except Exception as e:
                print(e)

        if clean_link is not None:
            try:
                if clean_link.endswith("/"):
                    clean_link = clean_link[:-1]
                clean_links.append(clean_link)
            except Exception as e:
                print(e)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))


def crawl(url, product):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = {url}

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress
        try:
            # Save text from the url to a <url>.txt file
            with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w") as f:

                # Get the text from the URL using BeautifulSoup
                soup = BeautifulSoup(requests.get(url).text, "html.parser")

                # Get the text but remove the tags
                text = soup.get_text()

                # If the crawler gets to a page that requires JavaScript, it will stop the crawl
                if ("You need to enable JavaScript to run this app." in text):
                    print("Unable to parse page " + url + " due to JavaScript being required")

                # Otherwise, write the text to the file in the text directory
                f.write(text)
        except Exception as e:
            print(e)  

        try:    
            # Get the hyperlinks from the URL and add them to the queue
            for link in get_domain_hyperlinks(local_domain, url):
                try:
                     if product in link and "search" not in link and link not in seen:
                        queue.append(link)
                        seen.add(link)
                except Exception as e:
                    print(e)          
        except Exception as e:
            print(e)



In [26]:
# import time
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urlparse, urljoin
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from collections import deque
# import os
# import re
# 
# # Selenium setup
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)
# 
# def get_hyperlinks(url):
#     driver.get(url)
#     time.sleep(2)  # Adjust based on page load time
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     hyperlinks = [urljoin(url, a['href']) for a in soup.find_all('a', href=True)]
#     return hyperlinks
# 
# HTTP_URL_PATTERN = r'^http[s]*://.+'
# 
# def get_domain_hyperlinks(local_domain, url):
#     clean_links = []
#     for link in set(get_hyperlinks(url)):
#         try:
#             if re.search(HTTP_URL_PATTERN, link):
#                 url_obj = urlparse


SyntaxError: incomplete input (3219616653.py, line 35)

In [28]:
# import time
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urlparse, urljoin
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from collections import deque
# import os
# import re
# 
# # Selenium setup
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)
# 
# def get_hyperlinks(url):
#     driver.get(url)
#     time.sleep(2)  # Adjust based on page load time
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     hyperlinks = [urljoin(url, a['href']) for a in soup.find_all('a', href=True)]
#     return hyperlinks
# 
# HTTP_URL_PATTERN = r'^http[s]*://.+'
# 
# def get_domain_hyperlinks(local_domain, url):
#     clean_links = []
#     for link in set(get_hyperlinks(url)):
#         try:
#             if re.search(HTTP_URL_PATTERN, link):
#                 url_obj = urlparse(link)
#                 if local_domain in url_obj.netloc:
#                     clean_links.append(link)
#             else:
#                 if link.startswith("/"):
#                     clean_links.append(urljoin(url, link))
#         except Exception as e:
#             print(f"Error processing link {link}: {e}")
# 
#     return list(set(clean_links))
# 
# def crawl(url, product):
#     local_domain = urlparse(url).netloc
#     queue = deque([url])
#     seen = {url}
# 
#     if not os.path.exists("text/"):
#         os.mkdir("text/")
#     if not os.path.exists(f"text/{local_domain}/"):
#         os.mkdir(f"text/{local_domain}/")
#     if not os.path.exists("processed"):
#         os.mkdir("processed")
# 
#     results = []
# 
#     while queue:
#         url = queue.pop()
#         print(f"Crawling URL: {url}")  # Debugging progress
#         try:
#             response = requests.get(url)
#             soup = BeautifulSoup(response.text, "html.parser")
#             text = soup.get_text()
# 
#             if "You need to enable JavaScript to run this app." in text:
#                 print(f"Unable to parse page due to JavaScript requirement: {url}")
#                 continue
# 
#             if product in url:
#                 results.append(url)
# 
#             with open(f'text/{local_domain}/{url[8:].replace("/", "_")}.txt', "w") as f:
#                 f.write(text)
# 
#         except Exception as e:
#             print(f"Error fetching {url}: {e}")
# 
#         try:
#             for link in get_domain_hyperlinks(local_domain, url):
#                 if link not in seen:
#                     queue.append(link)
#                     seen.add(link)
#         except Exception as e:
#             print(f"Error processing hyperlinks for {url}: {e}")
# 
#     return results
# 
# product = '1VCE8'
# urls = [f'https://www.zoro.com/search?q={product}']
# all_reviews = []
# 
# for url in urls:
#     reviews = crawl(url, product)
#     all_reviews.extend(reviews)
# 
# print("Product URLs found:", all_reviews)


Crawling URL: https://www.zoro.com/search?q=1VCE8
Crawling URL: https://www.zoro.com/test-instruments-gauges/c/24/
Crawling URL: https://www.zoro.com/borescope/c/7066/


KeyboardInterrupt: 

In [23]:
# import time
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# 
# # Selenium setup
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)
# 
# def search_product(product_id):
#     # Search URL for the product
#     search_url = f'https://www.zoro.com/search?q={product_id}'
#     driver.get(search_url)
#     time.sleep(2)  # Wait for the page to load
# 
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
# 
#     # Find the specific product link
#     product_link = None
#     for a in soup.find_all('a', href=True, class_='product-card-image__link'):
#         if product_id in a['href']:
#             product_link = urljoin(search_url, a['href'])
#             break
# 
#     if product_link:
#         print(f"Found product URL: {product_link}")
#     else:
#         print("Product URL not found.")
# 
#     return product_link
# 
# product_id = '1VCE8'
# product_url = search_product(product_id)
# 
# driver.quit()


Product URL not found.


In [25]:
# import time
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin
# 
# # Selenium setup
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)
# 
# def search_product(product_id):
#     # Search URL for the product
#     search_url = f'https://www.zoro.com/search?q={product_id}'
#     driver.get(search_url)
#     time.sleep(5)  # Wait for the page to load
# 
#     # Get the page source after it loads
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
# 
#     # Log the HTML content for debugging
#     with open('page_source.html', 'w', encoding='utf-8') as file:
#         file.write(soup.prettify())
# 
#     # Find the specific product link
#     product_link = None
#     for a in soup.select('a.product-card-image__link'):
#         if product_id in a['href']:
#             product_link = urljoin(search_url, a['href'])
#             break
# 
#     if product_link:
#         print(f"Found product URL: {product_link}")
#     else:
#         print("Product URL not found.")
# 
#     return product_link
# 
# product_id = '1VCE8'
# product_url = search_product(product_id)
# 
# driver.quit()


Product URL not found.


In [29]:
# import time
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin
# 
# # Selenium setup
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)
# 
# def search_product(product_id):
#     # Search URL for the product
#     search_url = f'https://www.zoro.com/search?q={product_id}'
#     driver.get(search_url)
#     time.sleep(5)  # Wait for the page to load
# 
#     # Get the page source after it loads
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
# 
#     # Find the specific product link
#     product_link = None
#     for a in soup.find_all('a', href=True):
#         if product_id in a['href']:
#             product_link = urljoin(search_url, a['href'])
#             break
# 
#     if product_link:
#         print(f"Found product URL: {product_link}")
#     else:
#         print("Product URL not found.")
# 
#     return product_link
# 
# product_id = '1VCE8'
# product_url = search_product(product_id)
# 
# driver.quit()


Found product URL: https://www.zoro.com/search?q=1VCE8&fqc%3Acategory=30#search-container


In [31]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Selenium setup
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def search_product(product_id):
    # Search URL for the product
    search_url = f'https://www.zoro.com/search?q={product_id}'
    driver.get(search_url)
    time.sleep(5)  # Wait for the page to load

    # Get the page source after it loads
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Find the specific product link using a precise CSS selector
    product_link = None
    for a in soup.select('a.product-card-image__link'):
        if product_id.lower() in a['href'].lower():
            product_link = urljoin(search_url, a['href'])
            break

    if product_link:
        print(f"Found product URL: {product_link}")
    else:
        print("Product URL not found.")

    return product_link

product_id = '1VCE9'
product_url = search_product(product_id)

driver.quit()


Found product URL: https://www.zoro.com/dayton-standard-duty-industrial-fan-30-non-oscillating-115vac-88009800-cfm-1vce9/i/G2998721/


In [41]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from bs4 import BeautifulSoup
# def extract_reviews(html_content):
#     if not html_content:
#         print("No HTML content to process.")
#         return []
# 
#     soup = BeautifulSoup(html_content, 'html.parser')
#     review_sections = soup.find_all('div', class_='pr-review pr-review-condensed')
# 
#     reviews_data = []
# 
#     for idx, review in enumerate(review_sections, start=1):
#         print(f"\nProcessing review {idx}:")
#         
#         # Extracting star rating
#         star_rating = review.find('div', class_='pr-rating-stars')
#         if star_rating and 'aria-label' in star_rating.attrs:  # Check if 'aria-label' exists in attrs
#             print("Star Rating:", star_rating['aria-label'])
#         else:
#             print("Star Rating not found.")
#             continue
#         
#         # Extracting review headline
#         review_headline = review.find('h5', class_='pr-rd-review-headline')
#         if review_headline:
#             print("Review Headline:", review_headline.text.strip())
#         
#         # Extracting submission details
#         submission_details = review.find('p', class_='pr-rd-author-submission-date')
#         if submission_details:
#             print("Submission Details:", submission_details.text.strip())
#         
#         # Extracting reviewer details
#         reviewer_details = review.find('p', class_='pr-rd-author-nickname')
#         if reviewer_details:
#             print("Reviewer Details:", reviewer_details.text.strip())
#         
#         # Extracting review text
#         review_text = review.find('p', class_='pr-rd-description-text')
#         if review_text:
#             print("Review Text:", review_text.text.strip())
#         
#         # Adding all data to reviews_data
#         reviews_data.append({
#             'Star Rating': star_rating['aria-label'].strip() if star_rating and 'aria-label' in star_rating.attrs else None,
#             'Review Headline': review_headline.text.strip() if review_headline else None,
#             'Submission Details': submission_details.text.strip() if submission_details else None,
#             'Reviewer Details': reviewer_details.text.strip() if reviewer_details else None,
#             'Review Text': review_text.text.strip() if review_text else None
#         })
# 
#     return reviews_data

def extract_reviews(html_content):
    if not html_content:
        print("No HTML content to process.")
        return []

    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extracting star rating
    star_rating_container = soup.find('section', class_='pr-review-snapshot-block-snippet')
    if star_rating_container:
        star_rating = star_rating_container.find('div', class_='pr-snippet-stars')
        star_rating_text = star_rating_container.find('div', class_='pr-snippet-rating-decimal').text.strip()
        if star_rating:
            star_rating_label = star_rating['aria-label']
            print(f"Star Rating: {star_rating_label}, {star_rating_text}")
        else:
            print("Star Rating not found.")
    else:
        print("Star Rating container not found.")

    # Extracting recommendation percentage
    recommendation_section = soup.find('section', class_='pr-review-snapshot-block-recommend')
    if recommendation_section:
        recommendation_percent = recommendation_section.find('span', class_='pr-reco-value').text.strip()
        print(f"Recommendation Percentage: {recommendation_percent}")
    else:
        print("Recommendation section not found.")

    # Extracting reviews
    reviews = soup.find_all('section', class_='pr-rd-content-block')
    reviews_data = []

    for idx, review in enumerate(reviews, start=1):
        print(f"\nProcessing review {idx}:")
        
        # Extracting review text
        review_text = review.find('p', class_='pr-rd-description-text')
        if review_text:
            print("Review Text:", review_text.text.strip())
        else:
            print("Review Text not found.")
        
        # Adding review data
        reviews_data.append({
            'Star Rating': star_rating_label if star_rating else None,
            'Rating Text': star_rating_text if star_rating_text else None,
            'Review Text': review_text.text.strip() if review_text else None
        })

    return reviews_data


# Selenium setup
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

def get_page_soup(url):
    print(f"Getting page soup for URL: {url}")
    driver.get(url)
    time.sleep(1)  # Wait for the page to load
    html = driver.page_source
    return BeautifulSoup(html, 'html.parser')

def search_product(product_id):
    print(f"Searching for product ID: {product_id}")
    search_url = f'https://www.zoro.com/search?q={product_id}'
    soup = get_page_soup(search_url)

    # Find the specific product link using a precise CSS selector
    for a in soup.select('a.product-card-image__link'):
        print(f"Checking link: {a['href']}")
        if product_id.lower() in a['href'].lower():
            product_url = urljoin(search_url, a['href'])
            print(f"Found product URL: {product_url}")
            return product_url

    print("Product URL not found.")
    return None

def find_reviews_link(product_url):
    print(f"Finding reviews link on product URL: {product_url}")
    soup = get_page_soup(product_url)

    # Find the reviews link using a precise CSS selector
    for a in soup.select('a'):
        print(f"Checking link: {a['href']}")
        if 'reviews' in a['href']:
            reviews_url = urljoin(product_url, a['href'])
            print(f"Found reviews URL: {reviews_url}")
            return reviews_url

    print("Reviews URL not found.")
    return None

def navigate_to_reviews(product_id):
    print(f"Navigating to reviews for product ID: {product_id}")
    product_url = search_product(product_id)
    if product_url:
        reviews_url = find_reviews_link(product_url)
        if reviews_url:
            print(f"Navigating to reviews URL: {reviews_url}")
            driver.get(reviews_url)
            time.sleep(1)  # Wait for the reviews page to load
            # Extract page source after page load
            html_content = driver.page_source
        
            # Pass HTML content to extract_reviews function
            reviews_data = extract_reviews(html_content)
            
            # Print extracted reviews data
            print("\nExtracted Reviews:")
            for review in reviews_data:
                print(review)
        else:
            print("Reviews URL not found.")
    else:
        print("Product URL not found.")

product_id = '1VCE8'
navigate_to_reviews(product_id)

driver.quit()


Navigating to reviews for product ID: 1VCE8
Searching for product ID: 1VCE8
Getting page soup for URL: https://www.zoro.com/search?q=1VCE8
Checking link: /dayton-standard-duty-industrial-fan-24-non-oscillating-115vac-38506200-cfm-1vce8/i/G1828802/
Found product URL: https://www.zoro.com/dayton-standard-duty-industrial-fan-24-non-oscillating-115vac-38506200-cfm-1vce8/i/G1828802/
Finding reviews link on product URL: https://www.zoro.com/dayton-standard-duty-industrial-fan-24-non-oscillating-115vac-38506200-cfm-1vce8/i/G1828802/
Getting page soup for URL: https://www.zoro.com/dayton-standard-duty-industrial-fan-24-non-oscillating-115vac-38506200-cfm-1vce8/i/G1828802/
Checking link: #main-content
Checking link: /safety/c/z1/
Checking link: /office-school-shipping-supplies/c/z9/
Checking link: /heating-cooling/c/30/
Checking link: /tools-machining/c/z2/
Checking link: /abrasives-polishers/c/1/
Checking link: /adhesives-tapes/c/4/
Checking link: /electrical-supplies/c/23/
Checking link: /ele

In [38]:
# import requests
# from bs4 import BeautifulSoup
# 
# def navigate_to_reviews(product_id):
#     url = f"https://www.grainger.com/product/{product_id}"
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }
#     
#     response = requests.get(url, headers=headers)
#     
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.content, 'html.parser')
#         
#         reviews_section = soup.find('section', {'id': 'pr-review-display', 'data-testid': 'review-list'})
#         
#         if reviews_section:
#             reviews = reviews_section.find_all('div', {'class': 'pr-review pr-review-condensed'})
#             
#             for review in reviews:
#                 review_text = {}
#                 
#                 # Extract headline
#                 headline = review.find('h5', {'class': 'pr-rd-review-headline'})
#                 if headline:
#                     review_text['headline'] = headline.text.strip()
#                 
#                 # Extract submission date
#                 submission_date = review.find('span', {'data-datetime': True})
#                 if submission_date:
#                     review_text['submission_date'] = submission_date.text.strip()
#                 
#                 # Extract author name
#                 author_name = review.find('span', {'class': 'pr-rd-author-nickname'})
#                 if author_name:
#                     review_text['author'] = author_name.text.strip().split('By ')[-1]
#                 
#                 # Extract review text
#                 review_body = review.find('p', {'class': 'pr-rd-description-text'})
#                 if review_body:
#                     review_text['review_text'] = review_body.text.strip()
#                 
#                 print(review_text)  # Print for debugging or store in a list/dictionary
#         else:
#             print("Reviews section not found on the page.")
#     else:
#         print(f"Failed to retrieve page with status code {response.status_code}")
# 
# # Example usage:
# product_id = "1VCE8"
# navigate_to_reviews(product_id)


Reviews section not found on the page.


In [35]:
# import requests
# from bs4 import BeautifulSoup
# 
# def navigate_to_reviews(url):
# 
#     try:
#         response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             print(f"Successfully connected to {url}")
#             return response.content
#         else:
#             print(f"Failed to connect to {url}. Status code: {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         return None
# 
# def extract_reviews(html_content):
#     if not html_content:
#         print("No HTML content to process.")
#         return
# 
#     soup = BeautifulSoup(html_content, 'html.parser')
#     review_sections = soup.find_all('div', class_='pr-review pr-review-condensed')
# 
#     reviews_data = []
# 
#     for idx, review in enumerate(review_sections, start=1):
#         print(f"\nProcessing review {idx}:")
#         
#         # Extracting star rating
#         star_rating = review.find('div', class_='pr-rating-stars')
#         if star_rating:
#             print("Star Rating:", star_rating['aria-label'])
#         
#         # Extracting review headline
#         review_headline = review.find('h5', class_='pr-rd-review-headline')
#         if review_headline:
#             print("Review Headline:", review_headline.text.strip())
#         
#         # Extracting submission details
#         submission_details = review.find('p', class_='pr-rd-author-submission-date')
#         if submission_details:
#             print("Submission Details:", submission_details.text.strip())
#         
#         # Extracting reviewer details
#         reviewer_details = review.find('p', class_='pr-rd-author-nickname')
#         if reviewer_details:
#             print("Reviewer Details:", reviewer_details.text.strip())
#         
#         # Extracting review text
#         review_text = review.find('p', class_='pr-rd-description-text')
#         if review_text:
#             print("Review Text:", review_text.text.strip())
#         
#         # Adding all data to reviews_data
#         reviews_data.append({
#             'Star Rating': star_rating['aria-label'].strip() if star_rating else None,
#             'Review Headline': review_headline.text.strip() if review_headline else None,
#             'Submission Details': submission_details.text.strip() if submission_details else None,
#             'Reviewer Details': reviewer_details.text.strip() if reviewer_details else None,
#             'Review Text': review_text.text.strip() if review_text else None
#         })
# 
#     return reviews_data
# # 
# # # Example usage:
# # def main():
# #     product_id = '1VCE8'
# #     html_content = navigate_to_reviews(product_id)
# # 
# #     if html_content:
# #         reviews_data = extract_reviews(html_content)
# #         print("\nExtracted Reviews:")
# #         for review in reviews_data:
# #             print(review)
# # 
# # if __name__ == "__main__":
# #     main()


Successfully connected to https://www.grainger.com/product/1VCE8

Extracted Reviews:


In [19]:
import requests
from bs4 import BeautifulSoup

product = '1VCE8'
urls = [f'https://www.zoro.com/search?q={product}']  # List of URLs obtained from the search engine
reviews = [crawl(url, product) for url in urls]
print(reviews)

https://www.zoro.com/search?q=1VCE8
https://www.zoro.com/search?q=1VCE8&fqc%3Acategory=30#search-container
https://www.zoro.com/search?q=1VCE8&fqc%3Acategory=30%2F4583#search-container
https://www.zoro.com/search?q=1VCE8&fqc%3Acategory=30%2F4583%2F4961#search-container
https://www.zoro.com/search?fqc%3Acategory=30%2F4583%2F4961&q=1VCE8
https://www.zoro.com/search?fqc%3Acategory=30%2F4583&q=1VCE8#search-container
https://www.zoro.com/search?fqc%3Acategory=30%2F4583%2F4961&q=1VCE8#search-container
https://www.zoro.com/search?fqc%3Acategory=30&q=1VCE8#search-container
https://www.zoro.com/search?fqc%3Acategory=30%2F4583&q=1VCE8
https://www.zoro.com/search?fqc%3Acategory=30&q=1VCE8
[None]
